## Building Research Agent tool

In [2]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os
import sendgrid
import asyncio
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [23]:
load_dotenv(override=True)

True

### We'd be utilizing below tools from SDK:

1. WebSearchTool
2. FileSearchTool
3. ComputerTool

In [4]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [8]:
message = "Latest AI Agent frameworks in 2025"

with trace('search'):
    result = await Runner.run(search_agent, message)
    display(Markdown(f"**Search Agent Result:** {result}"))

**Search Agent Result:** RunResult:
- Last agent: Agent(name="Search agent", ...)
- Final output (str):
    As of August 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.
    
    **LangChain** is a widely adopted open-source framework that enables developers to build applications powered by large language models (LLMs). Its modular design allows for the integration of various components, including prompt templates, agent modules for task automation, memory systems for context retention, and dynamic retrieval components for real-time data access. LangChain supports multiple LLMs and offers a rich ecosystem of tools, making it suitable for applications such as conversational agents, document analysis, and code generation. ([dev.to](https://dev.to/surgedatalab/best-5-frameworks-for-agentic-ai-in-2025-enabling-next-gen-intelligent-multi-agent-systems-40ce?utm_source=openai))
    
    **LangGraph**, built upon LangChain, focuses on creating stateful, graph-based AI agent systems that can incorporate human intervention. It employs a graph-based approach to define the workflow of prompts and agent interactions, offering explicit control over the flow of information and allowing for branching and debugging of complex agent behaviors. LangGraph is ideal for applications requiring complex customer service flows with escalation paths to human agents and guided decision-making systems. ([radarmagazine.com](https://www.radarmagazine.com/top-5-ai-agent-frameworks-it-executives-should-be-watching-in-2025/?utm_source=openai))
    
    **AutoGen**, developed by Microsoft, specializes in orchestrating multiple AI agents to form autonomous, event-driven systems capable of handling complex, multi-agent tasks seamlessly. It enables advanced multi-agent conversations, tool usage, and memory, and is tightly integrated with Microsoft's Semantic Kernel and Azure services for secure enterprise deployment. ([medium.com](https://medium.com/%40rajadityasatellite/2025-is-the-year-of-ai-agent-frameworks-cb24e3f9ffc7?utm_source=openai))
    
    **CrewAI** adopts a role-based agent collaboration approach, facilitating the creation of specialized agents that collaborate on complex projects akin to a team environment. It offers dynamic task planning, real-time performance monitoring, and orchestration of a variety of agents as distinct workers. CrewAI is suitable for complex projects requiring teamwork among agents, such as software development or project management. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))
    
    **Eliza** is an open-source, Web3-friendly AI agent operating system that seamlessly integrates with Web3 applications. It allows for reading and writing blockchain data, interacting with smart contracts, and is fully controlled by the user through regular TypeScript programming. Eliza emphasizes stable performance through pragmatic implementation of its runtime components. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))
    
    **Agent Lightning** is a flexible and extensible framework that enables reinforcement learning-based training of LLMs for any AI agent. It achieves complete decoupling between agent execution and training, allowing seamless integration with existing agents developed via diverse methods. Agent Lightning introduces a hierarchical RL algorithm, LightningRL, which contains a credit assignment module, enabling RL to handle complex interaction logic, such as multi-agent scenarios and dynamic workflows. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))
    
    **AutoAgent** is a fully-automated and zero-code framework for LLM agents, enabling users to create and deploy LLM agents through natural language alone. Operating as an autonomous Agent Operating System, AutoAgent comprises key components such as Agentic System Utilities, LLM-powered Actionable Engine, Self-Managing File System, and Self-Play Agent Customization module. It serves as a versatile multi-agent system for general AI assistants and has demonstrated effectiveness in generalist multi-agent tasks, surpassing existing state-of-the-art methods. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))
    
    These frameworks represent the forefront of AI agent development, each contributing to the evolution of intelligent, autonomous systems across various applications. 
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)

In [14]:
number_searches = 3 #To avoid getting charged for too many searches

INSTRUCTIONS = f"You are a helpful research assistant.  Given a query, come up with a set of web searches \
to perform to best answer the query. Output {number_searches} terms to query for."

class WebSearchItem(BaseModel):
    reason: str = Field(description="Why this search term is relevant to the query")
    query: str = Field(description="The search term to use")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description=f"A list of {number_searches} search terms to query for")


plan_agent = Agent(
    name="Planneragent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

# planner_agent = Agent(
#     name="PlannerAgent",
#     instructions=INSTRUCTIONS,
#     model="gpt-4o-mini",
#     output_type=WebSearchPlan,
# )

In [16]:
message = "Latest AI Agent frameworks in 2025"

with trace('search'):
    result = await Runner.run(plan_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most current frameworks available for AI agents in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To identify trends and new technologies in AI agent development for the year 2025.', query='AI agent development trends 2025'), WebSearchItem(reason='To explore industry standards and tools for creating AI agents in 2025.', query='popular AI agent tools frameworks 2025')]


In [25]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = os.getenv("YAHOO_ID")
    to_email = os.getenv("GMAIL_ID")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [26]:
send_html_email

FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales prospects', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_html_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x116765580>, strict_json_schema=True, is_enabled=True)